### 임베딩 모델 사용

##### 0. Blob 스토리지 생성 후, CORS 설정
##### 1. Blob 스토리지에 CSV 파일 업로드
##### 2. OpenAI
- 채팅 플레이 그라운드 '배포'탭 -> 모델 배포 -> 'text-embedding-ada-002' 선택 -> '사용자 정의'하여 분당 토큰 조절 -> 배포
##### 3. AI Search
- '개요' -> 데이터 가져오기(신규) -> Blob Storage -> RAG -> 스토리지 및 컨테이너 선택 -> OpenAI 리소스 및 text-embedding 모델 선택 -> 만들기 (인덱스가 생김)
##### 4. OpenAI
- 채팅 플레이 그라운드 - 데이터 추가 - 'Azure AI 검색' - 3번에 생성된 인덱스 선택 - '벡터 검색' 체크 - 임베딩 모델 선택 - 검색 유형 '하이브리드 + 의미체계', 기존 의미 체계 검색 구성 선택 - API 키 선택 
##### * Blob 스토리지에 데이터 추가 시, AI Search의 인덱서(인덱스X)에서, 초기화 및 재실행하여 추가된 데이터 기반 검색 가능

In [5]:
import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

# .env 환경변수 로드
load_dotenv()
endpoint = os.getenv("ENDPOINT_URL", "https://8ai022-openai.openai.azure.com/")                     # OpenAI Endpoint
deployment = os.getenv("AZURE_OAI_DEPLOYMENT", "gpt-4o-mini")                                       # OpenAI Model Name
subscription_key = os.getenv("AZURE_OAI_KEY", "REPLACE_WITH_YOUR_KEY_VALUE_HERE")                   # OpenAI Key
search_endpoint = os.getenv("SEARCH_ENDPOINT", "https://8ai022-ai-search.search.windows.net")       # AI-Search Endpoint
search_key = os.getenv("AZURE_SEARCH_KEY", "put your Azure AI Search admin key here")               # AI-Search Key
search_index = "rag-marge"

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    {
        "role": "system",
        "content": "주인님이 원하는 결과를 충실하게 답변하도록! 극극존칭 사용 필수!"
    },
    {
        "role": "user",
        "content": "MS AI School에 대해서 설명해줘."
    }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": f"{search_index}",
            "semantic_configuration": "rag-marge-semantic-configuration",
            "query_type": "vector_semantic_hybrid", # 하이브리드 + 의미체계
            "fields_mapping": {},
            "in_scope": True,
            #"role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.", #상단 chat_prompt에서 system role 작성했으므로, 지워주어야 함
            "filter": None,
            "strictness": 4,        # 엄격성
            "top_n_documents": 5,   # 검색된 문서
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            },
            "embedding_dependency": {
              "type": "deployment_name",
              "deployment_name": "text-embedding-ada-002"   # 텍스트 임베딩 모델
            }
          }
        }]
    }
)

print(completion.to_json())
print(completion.choices[0].message.context['citations'])
    

{
  "id": "3f904f87-8bde-4f6b-aaad-65be42f24cb5",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "MS AI School에 대해서 설명드리겠습니다. MS AI School은 Microsoft에서 제공하는 교육 프로그램으로, 인공지능(AI) 및 관련 기술에 대한 교육을 통해 참가자들이 AI의 기초부터 고급 내용까지 학습할 수 있도록 돕습니다. 이 프로그램은 다음과 같은 주요 요소로 구성됩니다:\n\n1. **교육 내용**: AI School에서는 Azure OpenAI, Azure Machine Learning, Azure Cognitive Services 등의 Microsoft AI 기술을 배우며, 실습을 통해 해당 기술을 실제로 적용할 수 있는 능력을 기릅니다 .\n\n2. **프로그램 구조**: 프로그램은 이론 교육과 실습을 포함하여, 참가자들이 AI 모델을 설계하고 구현하는 데 필요한 실용적인 기술을 습득할 수 있도록 구성되어 있습니다. 또한, 다양한 교육 자료와 리소스가 제공됩니다 [doc2].\n\n3. **참여 대상**: 이 프로그램은 AI에 관심이 있는 초보자부터 전문가까지 다양한 수준의 참가자를 대상으로 하며, 각자의 수준에 맞는 교육 과정을 선택할 수 있습니다 [doc1].\n\n4. **목표**: MS AI School의 궁극적인 목표는 참가자들이 AI의 원리를 이해하고, 이를 실제 비즈니스 문제 해결에 적용할 수 있도록 하는 것입니다 [doc5].\n\n이와 같은 내용으로 MS AI School은 Microsoft의 AI 기술을 배우고자 하는 모든 분들에게 유익한 교육 프로그램입니다. 추가적인 질문이 있으시다면 언제든지 말씀해 주시기 바랍니다!",
        "role": "assistant",
        "end_turn": true,
       